In [ ]:
import h5py
import os
import numpy as np
import pandas as pd
%matplotlib inline
import tensorflow as tf
from keras.models import Model
from joblib import Parallel, delayed
import joblib

## H5 Extraction functions

In [ ]:
N_ELECTRODES = 32
CUT_OFF = 120

In [ ]:
if os.path.exists("denoising_Dense_1.h5"):
    autoencoder = tf.keras.models.load_model('denoising_Dense_1.h5')
    output_layer = (autoencoder.layers[-11].output)
    encoder = Model(autoencoder.input, output_layer)
    encoder.summary()


In [ ]:
if os.path.exists("Classifier_cnn.h5"):
    model = tf.keras.models.load_model('Classifier_cnn.h5')

In [ ]:
with h5py.File('./Post TBI 2/2022-11-23T16-42-00_SpikeOnChip_SPOC1_Data_Clean.h5', "r+") as f:
    for i in range(N_ELECTRODES):
        print(f'\nNum electrode : {i}')
        dset = f[f'SpikeWindow-0.{i}']
        stmp = f[f'SpikeTimestamp-0.{i}']

        if(dset.shape[0] != 0):
            sp = np.empty((dset.shape[0],CUT_OFF),dset.dtype)

            for idx,spke in enumerate(dset):
                sp[idx] = (spke[0:CUT_OFF])
        
            test = pd.DataFrame(sp)
            test_encode = encoder.predict(test)
            predict = model.predict(test_encode).round()
            print('Results')
            print(f'Spike predicted : {np.count_nonzero(predict == 1)}')
            print(f'Noise predicted : {np.count_nonzero(predict == 0)}')

            res = np.where(predict == 1)[0].tolist()

            indx = 0
            test_spike_windows = np.empty((len(res),dset.shape[1]),dset.dtype)
            for idx,row in enumerate(dset):
                if(idx in res):
                    test_spike_windows[indx] = row
                    indx += 1

            indx = 0
            test_spike_timestamp = np.empty((len(res),stmp.shape[1]),stmp.dtype)
            for idx,row in enumerate(stmp):
                if(idx in res):
                    test_spike_timestamp[indx] = row
                    indx += 1

            del f[f'SpikeWindow-0.{i}']
            f.create_dataset(f'SpikeWindow-0.{i}',data=test_spike_windows)

            del f[f'SpikeTimestamp-0.{i}']
            f.create_dataset(f'SpikeTimestamp-0.{i}',data=test_spike_timestamp)
        else:
            print("No spike found on this electrode")
    f.close()
